In [1]:
from __future__ import print_function

from apiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from httplib2 import Http
from oauth2client import file, client, tools
import io

In [2]:
store = file.Storage('token.json')
creds = store.get()

In [17]:
service = build(serviceName='drive', version='v3', http=creds.authorize(Http()))

In [18]:
results = service.files().list( q="name contains 'Guia Anuro' " ).execute()

In [20]:
articlesFolderId = '0B0M5IL0AEOXiXzlWS0JKYWFpWFE'

In [7]:
def getFamiliesFoldersIds():
    results = service.files().list( q=f"'{articlesFolderId}' in parents and mimeType contains 'vnd.google-apps.folder'",
                                  fields='files(id,name)').execute().get('files')
    return  { r['name']: r['id'] for r in results }

def getFamilyDocuments( family, families_folders_ids ):
    family_folder_id = families_folders_ids.get(family)
    results = service.files().list( 
        q=f"'{family_folder_id}' in parents and mimeType contains 'document'",
        fields='files(id,name)').execute().get('files')
    return results

In [21]:
families_folders_ids = getFamiliesFoldersIds()

In [22]:
ids_to_download = [ f.get('id') for f in getFamilyDocuments( 'leptodactylidae', families_folders_ids ) ]

In [23]:
family_folders_ids = getFamiliesFoldersIds()

In [24]:
data = { family: [doc for doc in getFamilyDocuments(family, family_folders_ids)] for family, family_id in family_folders_ids.items()}

In [28]:
# Update species pages
basepath = '../especies/'

for family in data:
    for doc in [ doc for doc in data[family] if doc['name']!='familia' ]:
        docid = doc['id']
        species = doc['name']
        
        request = service.files().export_media(fileId=docid, mimeType='text/plain')
        with open(basepath+f'{species}.md', 'wb') as f:
            f.write(request.execute())

In [27]:
basepath = '../familias/'
for family in data:
    for doc in [doc for doc in data[family] if doc['name']=='familia']:
        docid=doc['id']
        
        request = service.files().export_media(fileId=docid, mimeType='text/plain')
        with open(basepath+f'{family}.md', 'wb') as f:
            f.write(request.execute())